In [3]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Support Vector Machines
## Classification Using SVM
Load dataset. We will use a toy dataset from sklearn.

In [139]:
from sklearn import datasets

#Load dataset
sklearn_dataset = datasets.load_breast_cancer()
Xx  = sklearn_dataset.data
y = sklearn_dataset.target * 2 - 1    # labels must be in {-1, 1} for the hinge loss
X = np.ones((Xx.shape[0], Xx.shape[1] + 1 ))    # add a column of ones for intercept
X[:, :-1] = Xx
print("(N, D) =", X.shape)

(N, D) = (569, 31)


## Prepare cost and prediction functions

In [67]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    hinge = 1 - y * (X @ w)
    hinge[hinge < 0] = 0
    
    reg = lambda_ / 2 * np.linalg.norm(w, 2)
    
    return hinge.sum() + reg

In [91]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    predictions = X @ w
    predictions = np.sign(predictions)
    
    #removing unlikey 0
    predictions[predictions == 0] = 1
    
    #accuracy is numer of correct classifications over total classifications
    accuracy = 1 - (np.abs(y - predictions) / 2)
    return accuracy.sum() / X.shape[0]

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [114]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    
    gradient_reg = lambda_ * w
    hinge = 1 - y_n * x_n * w
    gradient_hinge = -y_n * x_n
    gradient_hinge[hinge < 0] = 0
    return gradient_hinge + gradient_reg / num_examples

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [115]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 2 * int(1e5)
    gamma = 1e-4
    lambda_ = int(1e4)   # big because scales with N due to the formulation of the problem (not an averaged loss)
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))
    print(w)

sgd_for_svm_demo(y, X)

iteration=0, cost=42679.7148487474
iteration=10000, cost=435.91510975297797
iteration=20000, cost=445.5705533611234
iteration=30000, cost=442.4525066941727
iteration=40000, cost=447.14880468759225
iteration=50000, cost=441.4201114487252
iteration=60000, cost=441.5601205043288
iteration=70000, cost=444.5811191324051
iteration=80000, cost=442.0671074993543
iteration=90000, cost=445.1708324672238
iteration=100000, cost=443.274536293614
iteration=110000, cost=438.7468721089962
iteration=120000, cost=437.52522365687616
iteration=130000, cost=437.83580592158535
iteration=140000, cost=440.89415887951066
iteration=150000, cost=437.29109870745083
iteration=160000, cost=437.8683716068875
iteration=170000, cost=438.4867354273954
iteration=180000, cost=439.43814938034353
iteration=190000, cost=440.81132461498106
training accuracy = 0.6836555360281195
[ 3.44333161e-03  4.93682269e-03  1.05796541e-02 -7.52361866e-04
  3.57473220e-05  1.16050008e-05 -1.95957270e-05 -9.41006054e-06
  6.33019608e-05  2

## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [142]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    alpha[n] = lambda_ / ((x_n * x_n).sum() * y_n ** 2)
    if alpha[n] > 1 :
        alpha[n] = 1
    elif alpha[n] < 0: 
        alpha[n] = 0
    
    
    #update w
    w = (X.T * y) @ alpha / lambda_
    return w, alpha

In [145]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    k = X @ X.T
    yxxy = y * k * y
    return alpha.sum() - (1/(2 * lambda_)) * alpha.T @ yxxy @ alpha 

In [147]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 2*int(1e5)
    lambda_ = int(1e4)   # use same lambda as before in order to compare

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))
    print(alpha)

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:751.82934, dual:0.00905, gap:751.82029
iteration=10000, primal:144382.74005, dual:-2231.44510, gap:146614.18515
iteration=20000, primal:144382.74005, dual:-2231.44510, gap:146614.18515
iteration=30000, primal:144382.74005, dual:-2231.44510, gap:146614.18515
iteration=40000, primal:144382.74005, dual:-2231.44510, gap:146614.18515
iteration=50000, primal:144382.74005, dual:-2231.44510, gap:146614.18515
iteration=60000, primal:144382.74005, dual:-2231.44510, gap:146614.18515
iteration=70000, primal:144382.74005, dual:-2231.44510, gap:146614.18515
iteration=80000, primal:144382.74005, dual:-2231.44510, gap:146614.18515
iteration=90000, primal:144382.74005, dual:-2231.44510, gap:146614.18515
iteration=100000, primal:144382.74005, dual:-2231.44510, gap:146614.18515
iteration=110000, primal:144382.74005, dual:-2231.44510, gap:146614.18515
iteration=120000, primal:144382.74005, dual:-2231.44510, gap:146614.18515
iteration=130000, primal:144382.74005, dual:-2231.44510, gap:1